# Filtrado Colaborativo: *Non-negative Matrix Factorization*

El filtrado colaborativo basado en factorización matricial proporciona unos resultados excelentes en cuanto a calidad de predicciones y recomendaciones. Además, permite resolver el problema de la escalabilidad, puesto que, una vez aprendido el modelo, el cálculo de las predicciones se realiza en un tiempo ínfimo. 

Sin embargo, el algoritmo PMF tiene una deficiencia: la transformación de la matriz de votaciones en matrices de factores latentes es altamente abstracta para los usuarios y, por ende, imposibilita la explicación de las recomendaciones. Los modelos de KNN proporcionaban unas predicciones poco certeras y no escalaban bien, pero la explicación de dichas predicciones era muy sencilla: "*votarás este item con esta nota ya que estos usuarios que tienen intereses similares a los tuyos lo han votado así*". Sin embargo, PMF realiza las predicciones atendiendo a los factores latentes. ¿Qué significado tienen estos factores? ¿Qué quiere decir que un usuario tiene el factor 3 con un valor de 0,34? ¿Y que un item tiene el factor 8 con -4,06? Es complicado de explicar.

No poder explicar las recomendaciones provoca la desconfianza del usuario hacia el sistema. En general, los usuarios necesitan conocer el porqué de las cosas. Además, si se produce una predicción acertada y se explica dicha predicción, la satisfacción del usuario es doble. Igualmente, si se falla una predicción, pero se justifica correctamente, el usuario suele ser benevolente con el sistema.

La principal problematica de los factores latentes, más allá de su alto nivel de abstracción, es la inclusión de factores negativos dentro del modelo de fatorización matricial. Parece razonable justificar que las vataciones se encuentran condicionadas por una serie de factores / características / propiedades intrínsecas de los usuarios e items en un dominio concreto. Por ejemplo, indicar que te gustará una películas porque mezcla los genéros acción y comedia es una justificación comprensible. Sin embargo, tener factores con valores negativos hace imposible este tipo de justificaciones. ¿Que un factor valga 0,25 y otro valga -0,31 indica que el primero es afín y el segundo no? No necesariamente.

El modelo *NMF (**Non-negative Matrix Factorization**)*, al igual que otros algoritmos de factorización matricial, factoriza la matriz de votaciones $R$ en dos nuevas matrices $W$ y $H$ tales que verifiquen la siguiente expresión:

$$R \approx W \cdot H$$

De forma análoga a como sucedía en PMF, en esta expresión:

- $R$ representa la matriz (dispersa) con las votaciones de los usuarios (filas) a los items (columnas).
- $W$ representa las matriz (densa) de factores de los usuarios (filas) con los $k$ factores latentes (columnas).
- $H$ representa las matriz (densa) de factores de los items (columnas) con los $k$ factores latentes (filas).

Por lo tanto, el algoritmo NMF tratará de minimizar la diferencia cuadrática entre $R$ y $W \cdot H$, mediante la siguiente función de coste:

$$\sum_{u,i} (R_{u,i} - (W \cdot H)_{u,i})^2$$

Con el fin de facilitar las justificación de los resultados proporcionados por el modelo, añadimos una restricción adicional: **los valores de las matrices $W$ y $H$ deben ser siempre mayores o iguales a 0**.

La resolución de este problema podríamos plantearla mediante la ténica del descenso de gradiente, tal y como hicimos con PMF, sin embargo, la restricción de valores positivos permite transformar la función de coste para acelerar el proceso de aprendizaje. Para ello debemos considerar que esta función de coste es convexa únicamente en $W$ o en $H$, no en las dos funciones al mismo tiempo, por lo que se optimizará una de las matrices fijando la otra y, a continuación, se hará la operación inversa.

Partiendo de esta premisa, cuando estamos aplicando el método de descenso de gradiente, optimizar una matriz $A$ en la interación $n$ implica aplicar una actualización $U_n$ a los valores que la matriz $A$ tenía en la iteración $n-1$. En concreto, los valores de la actualización se corresponden con la inversa del gradiente de la función de coste.

$$A_n \leftarrow A_{n-1} + U_n $$

Sabiendo que en NMF los valores de esta matriz son siempre positivos podemos optar por optimizar la exponencial de la función de coste en lugar de la propia función de coste, ya que la función exponencial es una función monótona creciente.

Si definimos:

$$B = exp(A)$$

Podemos deducir que:

$$B_n = exp(A_n) = exp(A_{n-1} + U_n) = exp(A_{n-1}) \cdot exp(U_n) = B_{n-1} \cdot exp(U_n)$$

Aplicando este proceso a la función de coste utilizada por NMF, obtenemos las siguientes reglas de actualización de las matrices $W$ y $H$:

$$W \leftarrow W \cdot \frac{R \cdot H^T}{W \cdot H \cdot H^T}$$

$$H \leftarrow H \cdot \frac{W^T \cdot R}{W \cdot W^T \cdot H}$$




## Entrenamiento del modelo

Desarrollando las operaciones matriciales anteriores, podemos determinar las ecuación de actualización de los factores de los usuarios ($w_u$) y los items ($h_i$) que permiten optimizar las matrices $W$ y $H$ alternativamente hasta la convergencia.

La actualización del factor *k*-ésimo del usuario $u$ se hará de acuerdo a la siguiente ecuación:

$$w_{u,k} = w_{u,k} \cdot \frac{\sum_{i \in I_u} h_{i,k} \cdot r_{u,i}}{\sum_{i \in I_u} h_{i,k} \cdot \sum_{l=1}^K w_{u,l} * h_{i,l}}$$

Dónde $I_u$ denota los items votados por el usuario $u$ y $K$ simboliza el número de factores del modelo.

Por su parte, la actualización del factor *k*-ésimo del item $i$ se hará de acuerdo a la siguiente ecuación:

$$h_{i,k} = h_{i,k} \cdot \frac{\sum_{u \in U_i} w_{u,k} \cdot r_{u,i}}{\sum_{u \in U_i} w_{u,k} \cdot \sum_{l=1}^K w_{u,l} \cdot h_{i,l}}$$

Dónde $U_i$ denota los usuarios que han votado el item $i$ y $K$ simboliza el número de factores del modelo.

Podemos simplificar las ecuaciones anteriores puesto que la predicción del voto del usuario $u$ al item $i$ ($\hat{r}_{u,i}$) se calcula como el producto escalar de los factores del usuario $u$ y el item $i$:

$$\hat{r}_{u,i} = \sum_{k=1}^K w_{u,k} \cdot h_{i,k}$$

De este modo la actualización del factor *k*-ésimo del usuario $u$ se hará de acuerdo a la siguiente ecuación:

$$w_{u,k} = w_{u,k} \cdot \frac{\sum_{i \in I_u} h_{i,k} \cdot r_{u,i}}{\sum_{i \in I_u} h_{i,k} \cdot \hat{r}_{u,i}}$$

Mientras que la actualización del factor *k*-ésimo del item $i$ se hará de acuerdo a la siguiente ecuación:

$$h_{i,k} = h_{i,k} \cdot \frac{\sum_{u \in U_i} w_{u,k} \cdot r_{u,i}}{\sum_{u \in U_i} w_{u,k} \cdot \hat{r}_{u,i}}$$

Veamos cómo hacer esto con código.


## Carga del dataset

Para ilustrar mejor el funcionamiento el algoritmo NMF, vamos a desarrollar una implementación del mismo.

Para ello usaremos el dataset de [MovieLens 100K](https://grouplens.org/datasets/movielens/) que contiene 100.000 votos de 943 usuarios sobre 1682 películas. Este dataset ha sido dividido en votaciones de entrenamiento (80%) y votaciones de test (20%). Además, los códigos de usuarios e items, han sido modificados para que comience en 0 y terminen en el número de (usuarios / items) - 1.


Inicialmente definimos algunas constantes que nos serán necesarias durante la codificación del algoritmo:

In [0]:
import urllib
import random

In [0]:
NUM_USERS = 943
NUM_ITEMS = 1682

MIN_RATING = 1
MAX_RATING = 5

Y cargamos el dataset:

In [0]:
ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 

training_file = urllib.request.urlopen("https://drive.google.com/uc?export=view&id=1S4-sxOEvA3MDivaGf7iFirWqt1H6VtaH")
for line in training_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  ratings[int(u)][int(i)] = int(rating)

Del mismo modo, cargamos la matriz de votaciones de test:

In [0]:
test_ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 

test_file = urllib.request.urlopen("https://drive.google.com/uc?export=view&id=1LBgTF57DD2NA-petq_FaC1V-h7nrYIh9")
for line in test_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  test_ratings[int(u)][int(i)] = int(rating)

## Entrenamiento del modelo

Veamos cómo podemos aplicar las fórmulas anteriores para entrenar el modelo.


Primero, definimos los parámetros del modelo:

In [0]:
NUM_FACTORS = 5

Inicializamos los factores $w_u$ y $h_i$ con valores uniformes aleatorios en el intervalo \[0, 1].

In [0]:
w = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_USERS)] 
h = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)] 

Previo al entrenamiento del modelo, definimos una función que nos permite calcular las predicciones del mismo:

In [0]:
def compute_prediction (w_u, h_i):
  prediction = 0
  for k in range(NUM_FACTORS):
    prediction += w_u[k] * h_i[k]
  return prediction

Vamos a hacer que el modelo aprenda. Ejecutamos tantas veces como iteraciones haya la actualización de los factores. Para ello, recorremos el conjunto de votos y vamos haciendo las actualizaciones correspondientes.

In [0]:
NUM_ITERATIONS = 5

In [0]:
for it in range(NUM_ITERATIONS):
  print("Iteración " + str(it + 1) + " de " + str(NUM_ITERATIONS))
  
  # Actualizamos w fijando h
  
  for u in range(NUM_USERS):
    
    predictions = [None] * NUM_ITEMS
    
    for i in range(NUM_ITEMS):
        if ratings[u][i] != None:
          predictions[i] = compute_prediction(w[u], h[i])
    
    
    for k in range(NUM_FACTORS):
      
      sum_ratings = 0
      sum_predictions = 1e-10
      
      for i in range(NUM_ITEMS):
        if ratings[u][i] != None:
          
          sum_ratings += h[i][k] * ratings[u][i]
          sum_predictions += h[i][k] * predictions[i]
          
      w[u][k] = w[u][k] * sum_ratings / sum_predictions
          
    
  # Actualizamos h fijando w
  
  for i in range(NUM_ITEMS):
    
    predictions = [None] * NUM_USERS
    
    for u in range(NUM_USERS):
      if ratings[u][i] != None:
          predictions[u] = compute_prediction(w[u], h[i])
  
    for k in range(NUM_FACTORS):
      
      sum_ratings = 0
      sum_predictions = 1e-10
      
      for u in range(NUM_USERS):
        if ratings[u][i] != None:
          
          sum_ratings += w[u][k] * ratings[u][i]
          sum_predictions += w[u][k] * predictions[u]
          
      h[i][k] = h[i][k] * sum_ratings / sum_predictions

Iteración 1 de 5
Iteración 2 de 5
Iteración 3 de 5
Iteración 4 de 5
Iteración 5 de 5


## Cálculo de las predicciones

Como hemos comentado, calcular la predicción del voto del usuario *u* al item *i* implicar realizar el producto escalar de sus vectores de factores. Esta operación ha sido definida previamente.


## Cálculo de las recomendaciones

El cálculo de las recomendaciones, por lo general, simplemente implica seleccionar los *N* items con una predicción más alta. Por ejemplo, si quisiéramos recomendar *N = 3* items a un usuario que tuviera las siguientes predicciones:

|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| u 	|   	|  2,9 	|    	|  4,7 	|  5,0 	|    	|  1,2 	|    	|   	|  3,1 	|

Se le recomendarían a dicho usuario los items *i5*, *i4* e *i10*.


##Cálculo del MAE

En esta sección vamos a mostrar cómo calcular el error medio absoluto (MAE) de las predicciones realizadas por el algoritmo NMF.

Para ello, lo primero que debemos hacer es calcular las predicciones para todos los items que haya recibido una votación de test:

In [0]:
predictions = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 

for u in range(NUM_USERS):
  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None:
      predictions[u][i] = compute_prediction(w[u], h[i])

Y, a continuación, calculamos el MAE:

In [0]:
def get_user_mae (u):
  mae = 0
  count = 0
  
  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None and predictions[u][i] != None:
      mae += abs(test_ratings[u][i] - predictions[u][i])
      count += 1
  
  if count > 0:
    return mae / count
  else:
    return None

In [0]:
def get_mae ():
  mae = 0
  count = 0
  
  for u in range(NUM_USERS):
    user_mae = get_user_mae(u)
      
    if user_mae != None:
      mae += user_mae
      count += 1
  
  
  if count > 0:
    return mae / count
  else:
    return None   

In [0]:
mae = get_mae()
print("System MAE = " + str(mae))

System MAE = 0.8592044293164004


## Referencias

Lee, D. D., & Seung, H. S. (2001). **Algorithms for non-negative matrix factorization**. In Advances in neural information processing systems (pp. 556-562).


---

*Este documento ha sido desarrollado por **Fernando Ortega**. Dpto. Sistemas Informáticos, ETSI de Sistemas Informáticos, Universidad Politécnica de Madrid.*

*Última actualización: Abril de 2019*


<img src="https://drive.google.com/uc?export=view&id=1QuQDHyH_yrRbNt6sGzoZ8YcvFGEGlnWZ" alt="CC BY-NC">